## Greeting

In [1]:
print("Hello World!!")

Hello World!!


## Install tf.slim

In [2]:
!cd ~

In [5]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 44508 (delta 54), reused 45 (delta 2), pack-reused 44411
Receiving objects: 100% (44508/44508), 550.54 MiB | 48.73 MiB/s, done.
Resolving deltas: 100% (30378/30378), done.


In [6]:
! pip install contextlib2

In [7]:
import os

In [8]:
new_python_path = (os.environ.get("PYTHONPATH") or '') + ":models/research/slim"
%env PYTHONPATH = $new_python_path

env: PYTHONPATH=:models/research/slim


In [14]:
! echo $PYTHONPATH

:models/research/slim


In [22]:
os.chdir('./models/research/slim')

## Download datasets

In [ ]:
! python download_and_convert_data.py \
  --dataset_name=visualwakewords \
  --dataset_dir=data/visualwakewords

## Training

In [29]:
! python train_image_classifier.py \
    --train_dir = vww_96_grayscale \
    --dataset_name = visualwakewords \
    --dataset_split_name = train \
    --dataset_dir = data/visualwakeworkds \
    --model_name = mobilenet_v1_025 \
    --preprocessing_name = mobilenet_v1 \
    --train_image_size = 96 \
    --use_grayscale = True \
    --save_summaries_secs = 300 \
    --learning_rate = 0.045 \
    --label_smoothing = 0.1 \
    --learning_rate_decay_factor = 0.98 \
    --num_epochs_per_decay = 2.5 \
    --moving_average_decay = 0.9999 \
    --batch_size = 96 \
    --max_number_of_steps = 1000000

Traceback (most recent call last):
  File "train_image_classifier.py", line 22, in <module>
    import tf_slim as slim
ModuleNotFoundError: No module named 'tf_slim'


## Evaluating the Model

In [ ]:
! python eval_image_classifier.py \
    --alsologtostderr \
    --checkpoint_path = vww_96_grayscale/model.ckpt-698580 \
    --dataset_dir = data/visualwakewords \
    --dataset_name = visualwakewords \
    --dataset_split_name = val \
    --model_name=mobilenet_v1_025 \
    --preprocessing_name = mobilenet_v1 \
    --use_grayscale = True \
    --train_image_size = 96

## Exporting the Model to TensorFlow Lite

### Exporting to a GraphDef Protobuf File

In [ ]:
! python export_inference_graph.py \
    --alsologtostderr \
    --dataset_name = visualwakewords \
    --model_name = mobilenet_v1_025 \
    --image_size = 96 \
    --use_grayscale = True \
    --output_file = vww_96_grayscale_graph.pb

### Freezing the Weights

In [35]:
os.chdir('/')

In [ ]:
! git clone https://github.com/tensorflow/tensorflow
! python tensorflow/tensorflow/python/tools/freeze_graph.py \
    --input_graph = vww_96_grayscale_graph.pb \
    --input_checkpoint - vww_96_grayscale/model.ckpt-1000000 \
    --input_binary=true --output_graph = vww_96_grayscale_frozen.pb \
    --output_node_names = MobilenetV1/Predictions/Reshape_1